# Instagram Scraper

En el siguiente Notebook se desarrolla un programa con Selenium para hacer Scraping de la red social Instagram y generar una base de datos de publicaciones conformadas por (imagen y leyenda).

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [ ]:
import os
import wget
import time
import pandas as pd

En este Notebook se utilizará el navegador Chrome. Para poder automatizarlo, es necesario [descargar el driver](https://chromedriver.chromium.org/downloads) para la versión correspondiente y en la línea ```driver = webdriver.Chrome('/your/path/here')``` se añadirá la dirección en la que se encuentra el driver.

In [ ]:
# Crear sesión del navegador y abrir plataforma web de IG
driver = webdriver.Chrome('/your/path/here')
driver.get('https://www.instagram.com/')

In [ ]:
# Seleccionar los inputs de usuario y password
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# Vaciar los inputs
username.clear()
password.clear()

# Leer credenciales de archivo
with open('./local/ig_pw.txt', 'r') as f:
    pw = f.read()
    f.close()

# Ingresar las credenciales en la plataforma
username.send_keys("username")
password.send_keys(pw)

In [ ]:
# Iniciar sesión en IG
log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
time.sleep(1)

In [ ]:
# Saltar alertas
try:
    not_now = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))).click()
except:
    pass
try:
    not_now2 = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))).click()
except:
    pass

In [ ]:
# Buscar posts por hashtag
hashtag = "someHashtag"
driver.get("https://www.instagram.com/explore/tags/" + hashtag + "/")

In [ ]:
# Saltar el mensaje de obtener ayuda para temas sensibles
try:
    # Mostrar publicaciones
    WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='sqdOP  L3NKy    _8A5w5    '][contains(text(),'M')]"))).click()
except:
    pass

In [ ]:
# Abrir la primer publicación
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class='_9AhH0']"))).click()

In [ ]:
# Crear arreglo para almacenar los posts
posts_list = []

In [ ]:
# Número de posts a obtener
n_items = 100

# Ciclo que obtiene la imagen y el texto de cada publicación
for i in range(0, n_items):
    print('\n--- POST', i+1)
    post = []
    caption = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='C4VMK']/span"))).get_attribute("textContent")
    try:
        image = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, "//img[@class='FFVAD'][@srcset and not(@srcset='')]"))).get_attribute("src")
        post.append(image)
        post.append(caption)
        print(image)
        print(caption)
        posts_list.append(post)
    except:
        print("Post is a video")
        
    # Pasa al siguiente post
    driver.find_element_by_class_name("coreSpriteRightPaginationArrow").click()


In [ ]:
#Convertir el arreglo de publicaciones en un DataFrame
df = pd.DataFrame(posts_list, columns=['image', 'caption'])

# Guarda el DataFrame en archivo .csv
df.to_csv('instagram-posts.csv', index=False)

In [ ]:
df